# Image classification and change detection
A powerful application of remote sensing is detecting changes in land cover or land use. The natural environment changes from sudden and sometimes catastrophic events, but there is also a slower pace as in meandering rivers, or the annual cycle in vegetation. On longer time scales the environment changes due to gradual geomorphological changes and the effects of climete change. The built and managed environment also shows changes in land cover due to e.g. farming, dam building, city expansions, deforestation, or beach nourishment, to name a few.

Earth observation provides a fast and relatively cheap way to inventory and monitor these changes. A prerequisite for the successful application of remote sensing for change detection is that the changes in land cover result in changes in radiance values and that these radiance changes are large with respect to other causes of radiance changes such as changing atmospheric conditions, differences in soil moisture and differences in sun angle (Mas, 1999; Singh, 1989). Change detection methods can basically be subdivided into the following broad classes:

1. **Image differencing:** registered images acquired at different times are subtracted to produce a residual image which represents the change between the two dates. Pixels of no radiance change are distributed around the mean while pixels of radiance change are distributed in the tails of the statistical distribution.
2. **Vegetation Index Differencing:** The NDVI is calculated (the normalized ratio of red and near infrared reflectance) for the images acquired at different dates. Next the NDVI values of the two images are subtracted and result in high positive or high negative values for change areas.
3. **Direct multi-date classification:** This method is based on the single analysis of a combined dataset of the two dates images aiming at identifying areas of changes. Two images are first merged into one multi-band image set. Classes where changes are occurring are expected to present statistics significantly different from where change did not take place and can so be identified. Unsupervised classification methods e.g. the isolate algorithm to reveal the spectral distance patterns in the data.
4. **Post-classification analysis:** The two (or more) images acquired at different dates are first independently classified using the same legend (number of classes and type of classes). Next, the two images are overlaid and compared by subtracting or otherwise. It should be noted that the accuracy of the change detection depends on the accuracies of each of the individual classification results and the geometric match of the imagery.

# Visualise the full feature space for five land cover classes
Landsat's seven bands give a different spectral profile for water, sand, forest, urban and grassland. In this exercise, you will create the create new vector data by digitizing Areas Of Interest (AOIs) for these five classes and extract the pixel values from the landsat scene. The result will be visualized in a scatterplot matrix using seaborn. The exercise consists of the following steps:
1. Learn to digitize polygons in QGIS.
2. Digitize small uniform land cover units for water, sand, forest, urban and grassland. These are your AOIs.
3. Rasterize the AOI
4. Reformat the spectral data and the AOIs from raster format to a ```pandas``` dataframe.
5. Create a scatterplot matrix using Seaborn's ```pairplot```.

Step 1 can be completed by following along the QGIS [tutorial 5.1](https://docs.qgis.org/3.10/en/docs/training_manual/create_vector_data/create_new_vector.html). Use the 2015 landsat image as a background and use the RGB=543 band combination to visualize the satellite image. Make sure that the coordinate reference system of your shapefile equals the crs of the landsat image. For step 2, digitize around 20 AOIs, four four each land cover class. Give each feature two attributes: land cover and a numeric representation of the land cover:

| land_cover | value |
| --- | --- |
| water | 1 |
| sand | 2 |
| forest | 3 |
| grassland | 4 |
| urban | 5 |

Read the AOIs and visualise the data in a plot. If you are unable to do step 1 and 2, use the file 'aoi_NL_5_classes.shp' from the ```data``` folder

##### *Questions*:
1. Do you see any digitizing mistakes?
2. Do the raster and vector data share the same crs?

In [1]:
# Check the location of AOI (areas of interest (reference data))

aoi = gpd.read_file('../data/aoi_NL_5_classes.shp')
print (aoi.head())

fig,ax = plt.subplots(1,1, figsize=(10,10))
show(ndvi_2015, transform=ndvi_meta['transform'], ax=ax, cmap='gray', alpha=0.25)
aoi.plot(column='land_cover', legend=True, ax=ax, cmap='Set1')

NameError: name 'gpd' is not defined

## Rasterize the AOI
To rasterize the features in the shapefile, we will use ```rasterio```'s [rasterize](https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html) function. The shapes to be rasterized have to be an "(iterable of (geometry, value) pairs or iterable over) – geometries." We can obtain the shapes in this format from the GeoDataFrame using list comprehension. The number of rows and columns (out_shape) and the transform should be taken from the clipped raster dataset. 

In [ ]:
# Rasterize the aoi

from rasterio.features import rasterize

# Read the aoi file into a GeoDataFrame and extract the shapes as a list of tuples with (geometry, value) pairs
aoi_gdf = gpd.read_file('data/aoi_NL_5_classes.shp')
#aoi_gdf = gpd.read_file('data/aoi_NL_UTM.shp')
shapes = [(geom, value) for geom, value in zip(aoi_gdf['geometry'], aoi_gdf['class'])]

# Rasterize the vector data using the existing clipped raster as template
rs = rasterize(shapes,
               out_shape=ndvi_2015.shape[1:],
               transform=ndvi_meta['transform'],
               all_touched=True,
               dtype='float64')

# Show the rasterized data and write to file
fig,ax = plt.subplots(1,1, figsize=(10,10))
show(rs, transform=ndvi_meta['transform'], ax=ax)

with rasterio.open('scratch/aoi_all.tif', 'w', **ndvi_meta) as dst:
    dst.write_band(1,rs)